In [1]:
# 挂载云盘
import os
from google.colab import drive
drive.mount('/content/gdrive')

# 更改工程的运行文件夹
os.chdir("/content/gdrive/My Drive/Colab Notebooks/")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import io
import sys

def load_vectors(fname):
  fin = io.open(fname, 'r', encoding='utf-8', newline='\n', errors='ignore')
  n, d = map(int, fin.readline().split())
  print('Numbers:', n, 'Dimension:', d)
  data = {}
  for i, line in enumerate(fin):
    tokens = line.rstrip().split(' ')
    data[tokens[0]] = list(map(float, tokens[1:]))    
    if i == 10e4:
      break
  return data

pretrained_fastText_path = './Task2/wiki-news-300d-1M.vec'
wordMatrix = load_vectors(pretrained_fastText_path)


Numbers: 999994 Dimension: 300
内存大小： 5242976


In [0]:
wordMatrix.keys()

dict_keys([',', 'the', '.', 'and', 'of', 'to', 'in', 'a', '"', ':', ')', 'that', '(', 'is', 'for', 'on', '*', 'with', 'as', 'it', 'The', 'or', 'was', "'", "'s", 'by', 'from', 'at', 'I', 'this', 'you', '/', 'are', '=', 'not', '-', 'have', '?', 'be', 'which', ';', 'all', 'his', 'has', 'one', 'their', 'about', 'but', 'an', '|', 'said', 'more', 'page', 'he', 'your', 'will', 'its', 'so', 'were', 'had', 'also', 'only', 'if', 'time', 'some', 'people', 'like', 'who', 'them', 'other', 'they', 'when', 'Wikipedia', 'article', 'what', '#', 'just', '!', 'any', 'after', 'there', 'would', 'can', 'In', 'her', 'talk', 'use', 'then', 'into', 'up', '...', 'we', 'over', 'my', 'out', 'here', 'now', 'because', 'do', 'work', 'than', 'no', 'UTC', 'me', 'A', 'two', 'our', 'been', 'new', 'where', '–', 'first', 'such', 'made', '--', 'If', "'t", 'both', 'before', 'way', '1', 'through', 'information', 'him', 'being', 'many', 'most', 'But', 'those', 'while', 'name', 'This', 'should', 'how', 'even', 'these', 'see', 

In [0]:
import os, json

valid_row_data_path = './Task2/sna_data/sna_valid_author_raw.json'
valid_pub_data_path = './Task2/sna_data/sna_valid_pub.json'

valid_pub_data = json.load(open(valid_pub_data_path, 'r', encoding='utf-8'))
valid_data = json.load(open(valid_row_data_path, 'r', encoding='utf-8'))


### 数据预处理

In [0]:
import re
# 数据预处理

# 预处理名字
def precessname(name):   
    name = name.lower().replace(' ', '_')
    name = name.replace('.', '_')
    name = name.replace('-', '')
    name = re.sub(r"_{2,}", "_", name)
    return name

# 预处理机构,简写替换，
def preprocessorg(org):
    if org != "":
        org = org.replace('Sch.', 'School')
        org = org.replace('Dept.', 'Department')
        org = org.replace('Coll.', 'College')
        org = org.replace('Inst.', 'Institute')
        org = org.replace('Univ.', 'University')
        org = org.replace('Lab ', 'Laboratory ')
        org = org.replace('Lab.', 'Laboratory')
        org = org.replace('Natl.', 'National')
        org = org.replace('Comp.', 'Computer')
        org = org.replace('Sci.', 'Science')
        org = org.replace('Tech.', 'Technology')
        org = org.replace('Technol.', 'Technology')
        org = org.replace('Elec.', 'Electronic')
        org = org.replace('Engr.', 'Engineering')
        org = org.replace('Aca.', 'Academy')
        org = org.replace('Syst.', 'Systems')
        org = org.replace('Eng.', 'Engineering')
        org = org.replace('Res.', 'Research')
        org = org.replace('Appl.', 'Applied')
        org = org.replace('Chem.', 'Chemistry')
        org = org.replace('Prep.', 'Petrochemical')
        org = org.replace('Phys.', 'Physics')
        org = org.replace('Phys.', 'Physics')
        org = org.replace('Mech.', 'Mechanics')
        org = org.replace('Mat.', 'Material')
        org = org.replace('Cent.', 'Center')
        org = org.replace('Ctr.', 'Center')
        org = org.replace('Behav.', 'Behavior')
        org = org.replace('Atom.', 'Atomic')
        org = org.split(';')[0]  # 多个机构只取第一个
    return org

#正则去标点
def etl(content):
    content = re.sub("[\s+\.\!\/,;$%^*(+\"\')]+|[+——()?【】“”！，。？、~@#￥%……&*（）]+", " ", content)
    content = re.sub(r" {2,}", " ", content)
    return content

def get_org(co_authors, author_name):
    for au in co_authors:
        name = precessname(au['name'])
        name = name.split('_')
        if ('_'.join(name) == author_name or '_'.join(name[::-1]) == author_name) and 'org' in au:
            return au['org']
    return ''

### 聚类匹配

In [0]:
# 基于规则（按组织机构消歧） 线上得分：0.1165
from tqdm import tqdm
def disambiguate_by_org():
    res_dict = {}

    for author in tqdm(valid_data):
        res_dict[author] = []

        papers = valid_data[author]
        org_dict = {}
        org_dict_coauthor = {}
        no_org_list = []
        for p_id in papers:
            paper = valid_pub_data[p_id]
            authors = paper['authors']
            for paper_author in authors:
                name = precessname(paper_author['name'])
                org = preprocessorg(paper_author['org']) if 'org' in paper_author else ""
                name = name.split('_')

                if '_'.join(name) == author or '_'.join(name[::-1]) == author:    # 处理名字等价翻转的情况, eg. Kai Zhao/Zhao Kai
                    if org == "":
                        no_org_list.append((paper['id'], [precessname(paper_author['name']) for paper_author in authors]))
                    else:  # 按组织聚类
                        if org not in org_dict:
                            org_dict[org] = [paper['id']]
                            org_dict_coauthor[org] = [precessname(paper_author['name']) for paper_author in authors]
                        else:
                            org_dict[org].append(paper['id'])
                            org_dict_coauthor[org].extend([precessname(paper_author['name']) for paper_author in authors])

        # 没有组织的根据合作者交集
        for p_id, names in no_org_list:
            tmp = ""
            max_num = 1
            for org in org_dict_coauthor:
                set_a = set(names)
                set_b = set(org_dict_coauthor[org])
                intersection = set_a & set_b
                iou = len(intersection)
                if iou > max_num:
                    max_num = iou
                    tmp = org

            if max_num != 1:
                org_dict[tmp].append(p_id)
            else:
                res_dict[author].append([p_id])    # 如果没有共同作者，则大概率认为其为个人作者

        for org in org_dict:
            res_dict[author].append(org_dict[org])
        json.dump(res_dict, open('./Task2/results/disambiguate_by_org_result.json', 'w', encoding='utf-8'), indent=4)

disambiguate_by_org()


100%|██████████| 50/50 [00:37<00:00,  1.52it/s]


In [0]:
# 2. 基于规则（按合作者与组织机构消歧） 线上得分：0.2449
from tqdm import tqdm

def disambiguate_by_coauthor():
    res_dict = {}

    for author in tqdm(valid_data):
        res = []
        papers = valid_data[author]
        paper_dict = {}
        for p_id in papers:
            paper = valid_pub_data[p_id]
            d = {}
            authors = [precessname(paper_author['name']) for paper_author in paper['authors']]
            if author in authors:
                authors.remove(author)
            org = preprocessorg(get_org(paper['authors'], author))
            venue = paper['venue']
            d["authors"] = authors
            d["org"] = org
            d['keywords'] = paper['keywords'] if 'keywords' in paper else ""
            d['venue'] = venue

            if len(res) == 0:
                res.append([paper['id']])
            else:
                max_score = 0
                indx = 0
                for i, clusters in enumerate(res):
                    score = 0
                    for j, pid in enumerate(clusters):
                        insection = set(paper_dict[pid]['authors']) & set(authors)
                        score += len(insection)

                    if score > max_score:
                        max_score = score
                        indx = i

                if max_score > 0:
                    res[indx].append(paper['id']) # 若最高分大于0，将id添加到得分最高的簇中
                else:
                    res.append([paper['id']]) # 否则，另起一簇

            paper_dict[paper['id']] = d

        res_dict[author] = res
    json.dump(res_dict, open('./Task2/results/disambiguate_by_coauthor_result.json', 'w', encoding='utf-8'), indent=4)

disambiguate_by_coauthor()

100%|██████████| 50/50 [02:10<00:00,  3.45s/it]


In [0]:
# 3. 无监督聚类（根据合作者和机构TFIDF进行相似度聚类） 线上得分：0.2637
from tqdm import tqdm
from sklearn.cluster import DBSCAN
from sklearn.metrics.pairwise import pairwise_distances
from sklearn.feature_extraction.text import TfidfVectorizer

merge_data = {}
for author in valid_data:
    merge_data[author] = [valid_pub_data[paper_id] for paper_id in valid_data[author]]

def disambiguate_by_cluster():
    res_dict = {}
    for author in tqdm(merge_data):
        coauther_orgs = []
        papers = merge_data[author]
        if len(papers) == 0:
            res_dict[author] = []
            continue

        paper_dict = {}
        for paper in papers:
            authors = paper['authors']
            names = [precessname(paper_author['name']) for paper_author in authors]
            orgs = [preprocessorg(paper_author['org']) for paper_author in authors if 'org' in paper_author]  
            abstract = paper["abstract"] if 'abstract' in paper else ''
            coauther_orgs.append(etl(' '.join(names + orgs) + ' '+ abstract))         
        tfidf = TfidfVectorizer().fit_transform(coauther_orgs)
        # sim_mertric = pairwise_distances(tfidf, metric='cosine')
        
        clf = DBSCAN(min_samples=4)
        s = clf.fit_predict(tfidf)
        #每个样本所属的簇
        for label, paper in zip(clf.labels_, papers):
            if str(label) not in paper_dict:
                paper_dict[str(label)] = [paper['id']]
            else:
                paper_dict[str(label)].append(paper['id'])
        res_dict[author] = list(paper_dict.values())
    json.dump(res_dict, open('./Task2/results/disambiguate_by_DBSCAN_minkowski_result.json', 'w', encoding='utf-8'), indent=4)

disambiguate_by_cluster()

100%|██████████| 50/50 [00:40<00:00,  1.10it/s]
